In [ ]:
!pip install imagen-pytorch==1.24.2
from imagen_pytorch import Unet, SRUnet256, Imagen, ImagenTrainer
from imagen_pytorch.data import Dataset
from accelerate import notebook_launcher
import torch
import numpy as np
import pickle
import json
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files, drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
unet1 = Unet(
    dim = 192,
    dim_mults = (1, 2, 4),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True),
    layer_cross_attns = (False, True, True),
    use_linear_attn = True,
    channels = 3,
    channels_out = 3
)


unet2 = SRUnet256(
    dim = 192,
    dim_mults = (1, 2, 4),
    num_resnet_blocks = (2, 4, 8),
    layer_attns = (False, False, True),
    layer_cross_attns = (False, False, True),
    channels = 3,
    channels_out = 3
)


# imagen, which contains the unets above (base unet and super resoluting ones)
imagen = Imagen(
    unets = (unet1, unet2),
    condition_on_text = True,
    text_encoder_name = 't5-small',
    image_sizes = (16, 32),
    timesteps = 1000, # how many steps between gausian steps 1000 prob be better
    channels = 3,
).cuda()

trainer = ImagenTrainer(imagen).cuda()

In [ ]:
TRAINING_SET = ""
TRAINING_SET_LABELS = ""
LABEL_EMBEDDINGS = ""
PREVIOUS_CHECKPOINT = ""
FUTURE_CHECKPOINT = ""
FUTURE_CHECKPOINT_F = ""
unet = 1 #Should be 2 if PREVIOUS_CHECKPOINT is not empty

In [ ]:
gen_train_set = np.load(TRAINING_SET)
gen_train_labels = np.load(TRAINING_SET_LABELS)

In [ ]:
training_set = torch.from_numpy(gen_train_set).float()

In [ ]:
print(training_set.shape)
print(len(gen_train_labels))

torch.Size([50000, 3, 32, 32])
50000


In [ ]:
label_embeddings = np.load(LABEL_EMBEDDINGS)
print(label_embeddings.shape)

(10, 4, 512)


In [ ]:
if unet != 1:
  trainer.load(PREVIOUS_CHECKPOINT)

In [ ]:
batch_size = 32

In [ ]:
import matplotlib.pyplot as plt

low = 1
loss = []
for k in range(60):
  ix = np.arange(len(training_set))
  np.random.shuffle(ix)

  for j in range(len(ix)//batch_size):
    i = k*(len(ix)//batch_size) + j

    if len(loss) > 0 and loss[-1] <= low and i > 1000:
      low = loss[-1]
      print("----------------------------------------------------------------------")
      print(i, low)
      trainer.save(FUTURE_CHECKPOINT)

    if len(loss) > 0 and loss[-1] <= low:
      low = loss[-1]

    if i%1000 == 0 and i != 0:
      iii = np.random.randint(0, training_set.shape[0], 1)
      training_embeddings = label_embeddings[gen_train_labels[iii]]
      training_embeddings = torch.tensor(training_embeddings, requires_grad = True).float().cuda()

      samp = trainer.sample(batch_size = 1, text_embeds = training_embeddings, stop_at_unet_number = unet)
      if unet == 1:
        image = np.zeros((16, 16, 3))
      else:
        image = np.zeros((32, 32, 3))
      np_images = samp.cpu().detach().numpy()
      for j in range(3):
        image[:, :, j] = np_images[0, j, :, :]
      imgplot = plt.imshow(image)
      plt.show()
      print(gen_train_labels[iii])

    if i%100 == 0 and i != 0:
      print(i, loss[-1], low)
      
    jx = ix[j*batch_size:(j+1)*batch_size]
    training_images = training_set[jx].cuda()
    training_embeddings = label_embeddings[gen_train_labels[jx]]

    training_embeddings = torch.tensor(training_embeddings, requires_grad = True).float().cuda()

    loss.append(trainer(training_images, text_embeds = training_embeddings, unet_number = unet))
    trainer.update(unet_number = unet)
trainer.save(FUTURE_CHECKPOINT_F)